This notebook is dedicated to establishing a reliable method of transferring batches of data to and from Google Drive and storing metadata in an accessible format

John Marangola
11/2/2021

We begin by sketching out how data should be clearly and efficiently stored as follows:

In order to standardize on a simple and very useful convention, we define an enum for the pieces on the chess board. 

In [28]:
import pandas as pd
import numpy as np
from enum import Enum

class ChessPiece(Enum):
    PAWN = 1
    ROOK = 2
    KNIGHT = 3
    KING = 4
    QUEEN = 5
    BISHOP = 6
    EMPTY = 7
     
piece = ChessPiece.PAWN
if piece is ChessPiece.PAWN:
    print("This is a pawn!")
if piece != ChessPiece.KNIGHT:
    print("Not a knight!")




This is a pawn!
Not a knight!


In order to avoid remembering ambiguous conventions such as T/F for colors of square and color of piece (Which takes time to remember and makes de bugging hard), we use a similiar standard enum for colors of things (pieces and squares).

In [9]:
class Color(Enum):
    ORANGE = 1
    BLUE = 2
    BLACK = 3
    WHITE = 4

piece_1_color = Color.ORANGE
piece_2_color = Color.BLUE
print("pieces are opponents") if piece_1_color != piece_2_color else "pieces are allies"

pieces are opponents


Now we find a clear convention for labelling positions on the board. If you are unfamiliar with chess take a look at this image that visually explains so-called "algebraic" notation:


In [20]:
import urllib.request
from PIL import Image

urllib.request.urlretrieve(
  "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/SCD_algebraic_notation.svg/1200px-SCD_algebraic_notation.svg.png", "SCD_algebraic_notation.svg")
  
img = Image.open("SCD_algebraic_notation.svg")
img.show()

For the sake of simplicity, we will define positions as "LN" where L is the letter associated with the position and N is the number associated with the position ie:

In [25]:
position1 = "e2"
position1_alt = "E2"
position1_alt = position1_alt.lower()
print(f"automatic case convesion works: {position1 == position1_alt}")

position2 = "g1"
print(f"position2 equals position1: {position2 == position1}")

automatic case convesion works: True
position2 equals position1: False


This appears to be robust. Since the convention in chess is always <letter><number> it is illogical to even worry about things such as 2e and e2 not being equivalent. Now lets move on to the storing all the metadata for a single piece. We decided that the metadata fields that should be recorded for each image are:
    1. Piece type
    2. Piece color (or lack of)
    3. Position
    4. Color of tile

We can therefore define a function that recieves these fields as parameters: 

In [32]:

# (Skip type validation for now)
def print_metadata(piece_type, piece_color, position, tile_color):
    print(piece_type.name)
    print(f"piece color: {piece_color.name}")
    print(f"position: {position.lower()}")
    print(f"tile color: {tile_color.name}")

piece_color = Color.ORANGE
piece_type = ChessPiece.ROOK
position = "E5"
tile_color = Color.BLACK

print_metadata(piece_type, piece_color, position, tile_color)


ROOK
piece color: ORANGE
position: e5
tile color: BLACK


Clearly, we can never have any pieces other than {ROOK, KING, QUEEN, KNIGHT, ..., BISHOP} or the allowed colors. Everything is always in the correct format when saved and we will save space by only writing integers to the csv instead of numerous strings for instance:

In [37]:
demo_color = Color.BLACK
# "write" operation:
print(demo_color.value)
# Get the demo color back from # it is written as:
print(Color(3))


3
Color.BLACK


Since we need to store the metadata for many images, lets use pandas to organize this it in a way that is efficient! Heres a simple dataframe with random metadata:

In [76]:
import pandas as pd
import numpy as np
import random

# Can rescale metadata table for any general number of images
number_images = 10

cols = ["Piece Type", "Color", "Position", "Tile Color"]
rows = list(range(5))
data = np.random.randn(5, number_images)
df_temp = pd.DataFrame(
    {
        "Piece Type": [ChessPiece(random.randint(1, 7)).name for i in range(number_images)],
        "Piece Color": [Color(random.choice([1, 2])).name for i in range(number_images)],
        "Position" : [random.choice(list("abcdefgh")) + str(random.randint(1, 8)) for i in range(number_images)],
        "Tile Color" : [Color(random.randint(3, 4)).name for i in range(number_images)]
    }
)
df_temp

,Piece Type,Piece Color,Position,Tile Color
0,EMPTY,BLUE,a3,WHITE
1,BISHOP,BLUE,h6,WHITE
2,QUEEN,ORANGE,h3,WHITE
3,ROOK,BLUE,a3,WHITE
4,EMPTY,ORANGE,b3,BLACK
5,KNIGHT,ORANGE,h6,WHITE
6,EMPTY,ORANGE,c1,BLACK
7,QUEEN,ORANGE,g8,BLACK
8,QUEEN,ORANGE,h8,BLACK
9,PAWN,ORANGE,c6,BLACK


Yes, it is possible to have an empty square with a piece color... the above data is totally randomly generated and just quick and dirty to get an idea of how this would work. 

Moving on we should probably add the camera pose as well:

In [81]:
# For now, lets keep it basic
class Camera(Enum):
    BIRDSEYE = 1
    ANGLED = 2
    
df_temp = pd.DataFrame(
    {
        "Piece Type": [ChessPiece(random.randint(1, 7)).name for i in range(number_images)],
        "Piece Color": [Color(random.choice([1, 2])).name for i in range(number_images)],
        "Position" : [random.choice(list("abcdefgh")) + str(random.randint(1, 8)) for i in range(number_images)],
        "Tile Color" : [Color(random.randint(3, 4)).name for i in range(number_images)],
        "Camera": [Camera(random.randint(1, 2)).name for i in range(number_images)]
    }
)
df_temp

,Piece Type,Piece Color,Position,Tile Color,Camera
0,BISHOP,ORANGE,c3,BLACK,BIRDSEYE
1,EMPTY,BLUE,c1,BLACK,BIRDSEYE
2,QUEEN,ORANGE,d8,BLACK,BIRDSEYE
3,BISHOP,BLUE,f5,WHITE,ANGLED
4,BISHOP,BLUE,f6,WHITE,BIRDSEYE
5,KING,ORANGE,e2,WHITE,BIRDSEYE
6,PAWN,ORANGE,a2,BLACK,ANGLED
7,KING,BLUE,e5,WHITE,ANGLED
8,ROOK,ORANGE,g2,BLACK,ANGLED
9,ROOK,BLUE,b3,BLACK,BIRDSEYE


Looking good, now lets move on to uploading the metadata to Google Drive.

In [82]:
pip install pydrive

     |████████████████████████████████| 987 kB 3.2 MB/s 
     |████████████████████████████████| 197 kB 25.9 MB/s 
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27435 sha256=0a4ad07782da99ed5d30682c079fe0bb749ff7804873978a75f489ac0d844d58
  Stored in directory: /Users/johnmarangola/Library/Caches/pip/wheels/6e/98/e3/c91ae530a0508f87f1f24fe2b9df9d8e3952de1224d495e9e2
Successfully built pydrive
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Perform first time manual developer authentication using Oauth and pydrive, make sure Localhost:8080/ abnd Localhost:8090/ enabled

In [113]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth)

file1 = drive.CreateFile({'title': 'Hello.txt'})  # Create GoogleDriveFile instance with title 'Hello.txt'.
file1.SetContentString('Hello World!') # Set content of the file from given string.
file1.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1022961328214-bl4hn8614idt5sdup9996pk8rirkjf33.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


It worked!

Now lets create a test folder

In [118]:
def create_folder(folder):
    file1 = drive.CreateFile({'title': 'Hello.txt'})  # Create GoogleDriveFile instance with title 'Hello.txt'.
    file1.SetContentString('Hello World!') # Set content of the file from given string.
    file1.Upload()
    
create_folder("testFolder")